In [ ]:
import pandas as pd
import re
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from tqdm import tqdm
import gc

In [ ]:
# Загрузка модели и токенизатора
model_name = "UrukHan/t5-russian-spell"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.63M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def replace_numbers_with_placeholder(text):
    """Заменяет числа словами на PLACEHOLDER."""
    return re.sub(r'\b\d+\b', 'NUMBER', text)

In [ ]:
def restore_numbers_from_placeholder(text):
    """Восстанавливает числа из PLACEHOLDER (если это необходимо)."""
    return text.replace('NUMBER', '')  # Замена обратно на числа или оставить как есть

In [ ]:
# Функция для предварительной обработки текста
def preprocess_text(text):
    """Удаляет ненужные символы и приводит текст к нижнему регистру."""
    if isinstance(text, float):
        text = ''  # Заменяем float на пустую строку или используйте str(text) если хотите сохранить значение
    text = text.lower()  # Приведение к нижнему регистру
    text = text.replace('...', ' ')
    end_punctuation = ''
    if text.endswith('!'):
        end_punctuation = '.'
    text = text.rstrip('!')  # Удаление завершающего восклицательного знака
    text = text.replace('нрзб', '')  # Удаление 'нрзб'

    # Удаление всей пунктуации, кроме пробелов и дефисов
    text = re.sub(r'[^\w\s-]', '', text)

    # Восстановление точки в конце, если она была заменена
    if end_punctuation:
        text += end_punctuation

    return text

In [ ]:
# Функция для исправления текста с использованием ruT5
def correct_text(text):
    """Исправляет текст с помощью ruT5."""
    input_text = f"Исправь ошибки в тексте: {text}"
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs)
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text

In [ ]:
# Функция для выделения ошибок
def highlight_errors(original, corrected):
    """Выделяет ошибки в оригинальном тексте."""
    original_cleaned = preprocess_text(original)
    corrected_cleaned = preprocess_text(corrected)

    original_tokens = original_cleaned.split()
    corrected_tokens = corrected_cleaned.split()

    highlighted = []
    for token in original_tokens:
        if token in corrected_tokens:
            highlighted.append(token)
        else:
            highlighted.append(f'[{token}]')

    # Сохраняем точку в конце предложения, если она была
    if original and original[-1] in ['.', '!', '?']:
        highlighted.append('.')

    return ' '.join(highlighted)

In [ ]:
# Функция для обработки срезов
def process_slice(df_slice):
    df_slice['text'] = df_slice['text'].apply(preprocess_text)
    df_slice['corrected'] = df_slice['text'].apply(correct_text)
    df_slice['highlighted'] = df_slice.apply(lambda row: highlight_errors(row['text'], row['corrected']), axis=1)
    return df_slice[['text', 'corrected', 'highlighted']]

In [ ]:
# Чтение данных из файла
df = pd.read_csv('table.csv')  # Укажите ваш файл

In [ ]:
# Обработка данных с использованием срезов
results = []
for i in tqdm(range(0, len(df), 100), desc="Обработка данных"):
    df_slice = df[i:i + 100]  # Обработка срезов по 500 строк
    processed_slice = process_slice(df_slice)
    results.append(processed_slice)
    gc.collect()  # Очистка памяти

Обработка данных:   0%|          | 0/122 [00:00<?, ?it/s]<ipython-input-8-388738847b95>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slice['text'] = df_slice['text'].apply(preprocess_text)
<ipython-input-8-388738847b95>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_slice['corrected'] = df_slice['text'].apply(correct_text)
<ipython-input-8-388738847b95>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

KeyboardInterrupt: 

In [ ]:
# Объединение всех результатов
final_df = pd.concat(results)

# Сохранение результатов в файл
final_df.to_csv('highlighted_errors.csv', index=False)